# Supervised Summarization with T5

## Introduction

In this notebook, we will utilize Google's T5 model for summarizing the dialogues existing in the SAMSUM corpus. We will start with installing&loading the necessary packages/models/metrics etc, then move on with our analysis.

In [ ]:
!pip install transformers==4.4.2

!pip install datasets

!pip install py7zr

!pip install sentencepiece==0.1.94

!pip install rouge_score

!pip install Rouge

     |████████████████████████████████| 2.0MB 14.7MB/s 
     |████████████████████████████████| 901kB 43.1MB/s 
     |████████████████████████████████| 3.3MB 50.3MB/s 
     |████████████████████████████████| 225kB 14.2MB/s 
     |████████████████████████████████| 245kB 26.0MB/s 
     |████████████████████████████████| 112kB 25.5MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 358kB 16.1MB/s 
     |████████████████████████████████| 1.9MB 27.5MB/s 
     |████████████████████████████████| 122kB 48.6MB/s 
     |████████████████████████████████| 2.2MB 48.9MB/s 
     |████████████████████████████████| 1.1MB 14.8MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import nltk

import torch

import transformers
from datasets import load_dataset, list_datasets, load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

from rouge import Rouge

import string

#import spacy
#from spacy.lang.en.stop_words import STOP_WORDS
#from spacy.lang.en import English

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Let's load our dataset from the huggingface library directly, and take a brief look at its structure.

In [ ]:
samsum_dataset = load_dataset('samsum')

print(samsum_dataset['train'][0])
print('-'*50)
print(samsum_dataset)

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.
{'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)", 'id': '13818513', 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}
--------------------------------------------------
DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})


As we mentioned it in the report, the dataset comes with pre-separated train, test and validation datasets. This is really nice since we won't spend any extra time on splitting the original dataset.

We have also checked the first dialogue-summary pair above, looks like it is possible that the dialogues include some unnecessary characters such as '\r' or '\n'. On the other hand, the summary for the first dialouge of the train set looks fine, but we may need to check more of them later.

Let's look at the dialogues in validation and test datasets as well, this time in a more compact format.


In [ ]:
train = samsum_dataset['train']
val = samsum_dataset['validation']
test = samsum_dataset['test']

In [ ]:
train[0].keys()

dict_keys(['dialogue', 'id', 'summary'])

In [ ]:
pprint(train['dialogue'][:3], compact= True)
print('-'*50)
pprint(test['dialogue'][:3], compact= True)
print('-'*50)
pprint(val['dialogue'][:3], compact= True)

['Amanda: I baked  cookies. Do you want some?\r\n'
 'Jerry: Sure!\r\n'
 "Amanda: I'll bring you tomorrow :-)",
 'Olivia: Who are you voting for in this election? \r\n'
 'Oliver: Liberals as always.\r\n'
 'Olivia: Me too!!\r\n'
 'Oliver: Great',
 "Tim: Hi, what's up?\r\n"
 'Kim: Bad mood tbh, I was going to do lots of stuff but ended up '
 'procrastinating\r\n'
 'Tim: What did you plan on doing?\r\n'
 'Kim: Oh you know, uni stuff and unfucking my room\r\n'
 "Kim: Maybe tomorrow I'll move my ass and do everything\r\n"
 "Kim: We were going to defrost a fridge so instead of shopping I'll eat some "
 'defrosted veggies\r\n'
 'Tim: For doing stuff I recommend Pomodoro technique where u use breaks for '
 'doing chores\r\n'
 'Tim: It really helps\r\n'
 "Kim: thanks, maybe I'll do that\r\n"
 'Tim: I also like using post-its in kaban style']
--------------------------------------------------
["Hannah: Hey, do you have Betty's number?\n"
 'Amanda: Lemme check\n'
 'Hannah: <file_gif>\n'
 "Amanda: 

## Data Cleaning

All of the first three dialogues in our datasets consistently include the characters '\r' and '\n'. It would be best to clear them out from our text since they don't represent anything.

We will create three corresponding pandas dataframe files to show our data in a more neat form.


In [ ]:
df_train = pd.DataFrame()
df_val = pd.DataFrame()
df_test = pd.DataFrame()

for elem in ['id', 'dialogue', 'summary']:
  df_train[elem] = train[elem]
  df_val[elem] = val[elem]
  df_test[elem] = test[elem]

In [ ]:
for elem in ['dialogue', 'summary']:
  df_train[elem] = df_train[elem].str.replace('\r', ' ')
  df_train[elem] = df_train[elem].str.replace('\n', ' ')

  df_test[elem] = df_test[elem].str.replace('\r', ' ')
  df_test[elem] = df_test[elem].str.replace('\n', ' ')

  df_val[elem] = df_val[elem].str.replace('\r', ' ')
  df_val[elem] = df_val[elem].str.replace('\n', ' ')

In [ ]:
pd.set_option('display.max_colwidth', 500)

display(df_train.head())

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some? Jerry: Sure! Amanda: I'll bring you tomorrow :-),Amanda baked cookies and will bring Jerry some tomorrow.
1,13728867,Olivia: Who are you voting for in this election? Oliver: Liberals as always. Olivia: Me too!! Oliver: Great,Olivia and Olivier are voting for liberals in this election.
2,13681000,"Tim: Hi, what's up? Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating Tim: What did you plan on doing? Kim: Oh you know, uni stuff and unfucking my room Kim: Maybe tomorrow I'll move my ass and do everything Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores Tim: It really helps Kim: thanks, maybe I'll do that Tim: I also li...",Kim may try the pomodoro technique recommended by Tim to get more stuff done.
3,13730747,"Edward: Rachel, I think I'm in ove with Bella.. rachel: Dont say anything else.. Edward: What do you mean?? rachel: Open your fu**ing door.. I'm outside",Edward thinks he is in love with Bella. Rachel wants Edward to open his door. Rachel is outside.
4,13728094,"Sam: hey overheard rick say something Sam: i don't know what to do :-/ Naomi: what did he say?? Sam: he was talking on the phone with someone Sam: i don't know who Sam: and he was telling them that he wasn't very happy here Naomi: damn!!! Sam: he was saying he doesn't like being my roommate Naomi: wow, how do you feel about it? Sam: i thought i was a good rommate Sam: and that we have a nice place Naomi: that's true man!!! Naomi: i used to love living with you before i moved in ...","Sam is confused, because he overheard Rick complaining about him as a roommate. Naomi thinks Sam should talk to Rick. Sam is not sure what to do."


The text is not crystal clear but it looks like the unwanted characters are gone. We can move on with using the T5 now.

## Initial Trials with T5

Let's load our t5 tokenizer&model and check the output of tokenized sentences.

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
#device = torch.device('cpu')

In [ ]:
print(type(tokenizer)) # Sanity Check
print()
print(type(model)) # Sanity Check

<class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>

<class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>


In [ ]:
test_sent = "Amanda: I baked cookies. Do you want some? Jerry: Sure! Amanda: I'll bring you tomorrow :-)"
test_sent_2 = "Olivia: Who are you voting for "
pprint(tokenizer([test_sent,test_sent_2], max_length=100, truncation=True), compact = True)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                     1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[21542, 10, 27, 13635, 5081, 5, 531, 25, 241, 128, 58, 16637, 10,
                10625, 55, 21542, 10, 27, 31, 195, 830, 25, 5721, 3, 10, 18, 61,
                1],
               [25051, 10, 2645, 33, 25, 10601, 21, 1]]}


Here we define a function that will transform our dataset dictionary files directly for the summarization task. It will take text inputs from both the dialogues and summaries and clean them before tokenizing.

It eventually returns a dictionary consisting of dialogue token ids, their attention mask and the summary token ids.

In [ ]:
max_input_length = 512
max_target_length = 512
prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + (doc.replace('\r', ' ').replace('\n', '')) for doc in examples["dialogue"]]
    #print(inputs)
    #print(len(inputs[0]))
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    inputs_summary = [(doc.replace('\r', ' ').replace('\n', '')) for doc in examples["summary"]]
    #print(inputs_summary)
    #print(len(inputs_summary[0]))
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(inputs_summary, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(train[:2])

{'input_ids': [[21603, 10, 21542, 10, 27, 13635, 5081, 5, 531, 25, 241, 128, 58, 16637, 10, 10625, 55, 21542, 10, 27, 31, 195, 830, 25, 5721, 3, 10, 18, 61, 1], [21603, 10, 25051, 10, 2645, 33, 25, 10601, 21, 16, 48, 4356, 58, 15865, 10, 18587, 7, 38, 373, 5, 25051, 10, 1212, 396, 1603, 15865, 10, 1651, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[21542, 13635, 5081, 11, 56, 830, 16637, 128, 5721, 5, 1], [25051, 11, 20373, 5144, 33, 10601, 21, 10215, 7, 16, 48, 4356, 5, 1]]}

Let's map this function to the whole samsum dataset.

In [ ]:
tokenized_datasets = samsum_dataset.map(preprocess_function, batched=True)

Lets also define the arguments for our model too. We will evaluate our performance per epochs and use a batch size of 16. The important arguments here are the number of training epochs, batch size and learning rate; with the first one being probably the most important among all.

In [ ]:

batch_size = 16
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Let's define another function for that will compute the rouge score during training and return the results.

In [ ]:
metric = load_metric("rouge")

def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = Rouge().get_scores(decoded_preds, decoded_labels, avg = True)
    for k,v in result.items():
      result = {k: v['f'] * 100}

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

We can finally start the training. We will train for one epoch only on the validation data.

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Runtime,Samples Per Second
1,2.013600,1.866449,39.096700,17.224000,33.016300,36.421900,16.497600,71.594600,11.425000


TrainOutput(global_step=921, training_loss=2.030467477605863, metrics={'train_runtime': 345.7049, 'train_samples_per_second': 2.664, 'total_flos': 2417202840772608.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 4040153, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 63278290, 'init_mem_gpu_peaked_delta': 484207616, 'train_mem_cpu_alloc_delta': 314772, 'train_mem_gpu_alloc_delta': 484107776, 'train_mem_cpu_peaked_delta': 66885090, 'train_mem_gpu_peaked_delta': 5353555968})

### Second Run

Let's increase the number of training epochs to see if the results change significantly or not. If they do, we will conduct extensive tuning for the other arguments(such as learning rate); if they don't, we won't go down that path.

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
metric = load_metric("rouge")
def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = Rouge().get_scores(decoded_preds, decoded_labels, avg = True)
    final_result = {}
    for k,v in result.items():
      if (k == 'rouge-1') or (k == 'rouge-2') or (k == 'rouge-l'):
        final_result[k] = round(v['f'],4)
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    # Uncomment these cells below if you'd like to compare predictions and true
    # values manually.
     
    #print(decoded_preds[:2])
    #print(decoded_labels[:2])

    return final_result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Runtime,Samples Per Second
1,1.847700,1.802188,0.376400,0.168900,0.375400,70.740700,11.563000
2,1.949400,1.785100,0.381200,0.175300,0.380500,70.809000,11.552000
3,1.917600,1.769705,0.386000,0.178100,0.383500,70.743600,11.563000
4,1.899300,1.761391,0.382700,0.177100,0.379700,70.672500,11.575000
5,1.876200,1.753501,0.382700,0.175200,0.379200,70.818900,11.551000
6,1.865100,1.750863,0.383200,0.177600,0.380600,71.150000,11.497000
7,1.862500,1.745924,0.388200,0.181100,0.385700,71.220400,11.485000
8,1.838600,1.740326,0.389500,0.179900,0.385700,71.244600,11.482000
9,1.842400,1.740314,0.389100,0.181000,0.386200,71.201700,11.488000
10,1.842000,1.738942,0.388100,0.180300,0.385700,71.468300,11.446000


TrainOutput(global_step=9210, training_loss=1.8777457702172824, metrics={'train_runtime': 3462.6253, 'train_samples_per_second': 2.66, 'total_flos': 2.403353891050291e+16, 'epoch': 10.0, 'init_mem_cpu_alloc_delta': 176883, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 909964, 'train_mem_gpu_alloc_delta': 484111872, 'train_mem_cpu_peaked_delta': 68787832, 'train_mem_gpu_peaked_delta': 5353551872})

The results did not change much on the validation set even if we increase our training epochs from 1 to 10. As a consequence, we won't be tuning our arguments extensively. But we still need to check our model's performance on the test set.

## Test Set Results

From the results that we obtained in the validation set, we can see that there is no golden epoch number that optimizes f1 scores for all of the rouge metrics. We will pick 7 for the number of epochs since it maximizes the rouge-2 score, while doing very well with the other two at the same time.

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=7,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = Rouge().get_scores(decoded_preds, decoded_labels, avg = True)
    final_result = {}
    for k,v in result.items():
      if (k == 'rouge-1') or (k == 'rouge-2') or (k == 'rouge-l'):
        final_result[k] = round(v['f'],4)
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    # Uncomment these cells below if you'd like to compare predictions and true
    # values manually.
    
    #print(decoded_preds[:2])
    #print(decoded_labels[:2])

    return final_result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Runtime,Samples Per Second
1,1.675900,1.738670,0.388700,0.168300,0.378900,74.616400,10.976000
2,1.791400,1.733005,0.389000,0.170300,0.381900,77.489600,10.569000
3,1.779900,1.728448,0.388100,0.172400,0.382800,75.108000,10.904000
4,1.773600,1.725130,0.388600,0.172900,0.383000,77.423000,10.578000
5,1.763900,1.724511,0.389800,0.172800,0.384600,77.099700,10.623000


["Betty's number is Lemme.\nHannah and Larry are at the park together.", "Rob and Eric are going to watch some of Eric's stand-ups on youtube"]
["Hannah needs Betty's number but Amanda doesn't have it.\nShe needs to contact Larry.", 'Eric and Rob are going to watch a stand-up on youtube.']
["Betty's number is Lemme.\nAmanda asks Larry to call her last", "Rob and Eric are going to watch some of Eric's stand-ups on youtube"]
["Hannah needs Betty's number but Amanda doesn't have it.\nShe needs to contact Larry.", 'Eric and Rob are going to watch a stand-up on youtube.']
["Betty's number is Lemme.\nAmanda asks Larry to call her last", "Rob and Eric are going to watch some of Eric's stand-ups on youtube"]
["Hannah needs Betty's number but Amanda doesn't have it.\nShe needs to contact Larry.", 'Eric and Rob are going to watch a stand-up on youtube.']
["Betty's number is Lemme.\nAmanda asks Larry to call her last", "Rob and Eric are going to watch some of Eric's stand-ups on youtube"]
["Hanna

Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l,Runtime,Samples Per Second
1,1.675900,1.738670,0.388700,0.168300,0.378900,74.616400,10.976000
2,1.791400,1.733005,0.389000,0.170300,0.381900,77.489600,10.569000
3,1.779900,1.728448,0.388100,0.172400,0.382800,75.108000,10.904000
4,1.773600,1.725130,0.388600,0.172900,0.383000,77.423000,10.578000
5,1.763900,1.724511,0.389800,0.172800,0.384600,77.099700,10.623000
6,1.763300,1.724133,0.388500,0.173500,0.383200,76.451400,10.713000
7,1.766700,1.721247,0.391100,0.174200,0.384900,76.059900,10.768000


["Betty's number is Lemme.\nAmanda asks Larry to call her last", "Rob and Eric are going to watch some of Eric's stand-ups on youtube"]
["Hannah needs Betty's number but Amanda doesn't have it.\nShe needs to contact Larry.", 'Eric and Rob are going to watch a stand-up on youtube.']
["Betty's number is Lemme.\nAmanda asks Larry to call her last", "Rob and Eric are going to watch some of Eric's stand-ups on youtube"]
["Hannah needs Betty's number but Amanda doesn't have it.\nShe needs to contact Larry.", 'Eric and Rob are going to watch a stand-up on youtube.']


TrainOutput(global_step=6447, training_loss=1.7646830307155945, metrics={'train_runtime': 2475.1679, 'train_samples_per_second': 2.605, 'total_flos': 1.6828091617714176e+16, 'epoch': 7.0, 'init_mem_cpu_alloc_delta': 221470, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 596842, 'init_mem_gpu_peaked_delta': 14651034112, 'train_mem_cpu_alloc_delta': 608660, 'train_mem_gpu_alloc_delta': 484167168, 'train_mem_cpu_peaked_delta': 68451250, 'train_mem_gpu_peaked_delta': 5349498880})

The output is a bit distorted(we tried it for two times to get a nicer looking one) but colab managed to complete running the model till the end. 

The Rouge scores are not the best one can achieve, but they are not too bad either, considering the metrics we saw in the research papers that dealt with dialogue summarization. The task itself is quite challenging but the results can be improved given more resources and time.